In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
from helperToolz.mirmazloumi import *
from pympler import asizeof
from other_repos.pyTSEB.pyTSEB import meteo_utils
from other_repos.pyTSEB.pyTSEB import resistances
from other_repos.pyTSEB.pyTSEB import net_radiation
from other_repos.pyTSEB.pyTSEB import clumping_index 
from other_repos.pyTSEB.pyTSEB import TSEB
import rasterio

In [ ]:
# this might dock onto the sharpener process??

In [4]:
# set the year, month and day to estimate evapotranspiration for
year = 2019
month = 'July'
day = 1
comp = 'minVZA'
tempOut = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/'

# path to era5 raw data
era5_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/'

# the LST acquisition time should determine which sharpened LST files are associatedto be processed (as they are associated with it)
LST_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/{year}/Daily_AcqTime_{comp}_{year}_{month}.tif' # epsg 4326

# the VZA at the time of LST acquisition is need
VZA_at_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/VZA/comp/{comp}/{year}/Daily_VZA_{comp}_{year}_{month}.tif' # epsg 4326

# the DEM, SLOPE, ASPECT, LAT, LON will be used to sharpen some of the era5 variables (the the resolution of the DEM)
dem_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/DEM_GER_LST_WARP.tif' # epsg 4326
slope_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/SLOPE_GER_LST_WARP.tif' # epsg 4326
aspect_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/ASPECT_GER_LST_WARP.tif' # epsg 4326
lat_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LAT_GER_LST_WARP.tif' # epsg 4326
lon_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LON_GER_LST_WARP.tif' # epsg 4326

# the geopotential is needed for the sharpening as well
geopot_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/tiff/low_res/geopotential/geopotential_low_res.tif' # epsg 4326

# for the resampling to S2, we need a template --> sharpened LST
LST_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/{comp}_{year}_{month}_{day:02d}_mvwin50_cv15_regrat0.3.tif' # epsg:3035

# for NDVI calculation (estimating LAI and others), we use the S2 composite used for sharpening
S2_file = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/tempDump/S2_20190705/Force_X_from_65_to_73_Y_from_39_to_47/Force_X_from_65_to_73_Y_from_39_to_47_Cube.vrt'

# find era5 file that matches the month of LST observation
valid_variables = sorted(list(dict.fromkeys(file.split('/')[-2] for file in getFilelist(era5_path, '.grib', deep=True) \
                                   if not any(var in file for var in ['geopotential', 'total_column_water_vapour']))))

# get a list for those era5 files that match the year and month of the provided LST acquisition file
era5_path_list = find_grib_file(getFilelist(era5_path, '.grib', deep=True), LST_acq_file)
era5_path_list = [path for path in era5_path_list if any(variable in path for variable in valid_variables)] # era5 are epsg 4326 and still will be after warping to doy
temp_pressure_checker(era5_path_list)

temperature will be processed before surface pressure - continue


In [5]:
# warp datasets needed for calculations to the spatial extent of the sharpened LST
LST_acq_spatial_sub = warp_raster_to_reference(source_path=LST_acq_file, reference_path=S2_file, output_path='MEM', resampling='near', keepRes=True)
VZA_at_acq_file_sub = warp_raster_to_reference(source_path=VZA_at_acq_file, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
dem_sub = warp_raster_to_reference(source_path=dem_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
slope_sub  = warp_raster_to_reference(source_path=slope_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
aspect_sub = warp_raster_to_reference(source_path=aspect_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
lat_sub = warp_raster_to_reference(source_path=lat_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
lon_sub = warp_raster_to_reference(source_path=lon_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
geopot_sub = warp_raster_to_reference(source_path=geopot_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)

In [6]:
# load the era5 variable into cache at LST resolution and read-in the modelled times (one time step per band)
for path in era5_path_list:
    print(f'processing {path}')
    # check if DEM sharpener needs to be applied
    if '100m_u_component_of_wind' in path:
        # do the warping without sharpening
        wind100_u = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, lst_acq_file=LST_acq_spatial_sub, doy=day)

    elif '100m_v_component_of_wind' in path:
               # do the warping without sharpening
        wind100_v = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, lst_acq_file=LST_acq_spatial_sub, doy=day)

    # elif 'geopotential' in path:
    #     # do the warping without sharpening
    #     geopot = get_warped_ERA5_at_doy(path_to_era_grib=path, lst_acq_file=LST_acq_file, doy=day)

    elif 'downward' in path: # terrain correction included
        ssrd, szenith, sazimuth = get_ssrdsc_warped_and_corrected_at_doy(path_to_ssrdsc_grib=path, reference_path=LST_acq_spatial_sub, 
                                                                         lst_acq_file=LST_acq_spatial_sub, doy=day, 
                                                                         slope_path=slope_sub,
                                                                         aspect_path=aspect_sub,
                                                                         dem_path=dem_sub,
                                                                         lat_path=lat_sub,
                                                                         lon_path=lon_sub)

    elif '2m_temperature' in path: # DEM and adiabatic sharpening, following Guzinski 2021
        air_temp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                          lst_acq_file=LST_acq_spatial_sub, doy=day,
                                          sharp_blendheight=100,
                                          sharp_DEM=dem_sub,
                                          sharp_geopot=geopot_sub,
                                          sharp_rate=STANDARD_ADIABAT,
                                          sharpener='adiabatic')

    elif '2m_dewpoint_temperature' in path: # DEM and adiabatic sharpening, following Guzinski 2021
        dew_temp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                          lst_acq_file=LST_acq_spatial_sub, doy=day,
                                          sharp_blendheight=100,
                                          sharp_DEM=dem_sub,
                                          sharp_geopot=geopot_sub,
                                          sharp_rate=MOIST_ADIABAT,
                                          sharpener='adiabatic')

    else: 
        # do warping with DEM sharpening only
        # sanity check
        if not 'surface_pressure' in path:
            raise ValueError('There is and unattended ERA5 variable in the loop - CHECK!!!!')
        else:
            sp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                        lst_acq_file=LST_acq_spatial_sub, doy=day,
                                        sharp_DEM=dem_sub,
                                        sharp_blendheight=100,
                                        sharp_geopot=geopot_sub,
                                        sharp_temp=air_temp,
                                        sharpener='barometric')

processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/100m_u_component_of_wind/100m_u_component_of_wind_2019_7.grib
processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/100m_v_component_of_wind/100m_v_component_of_wind_2019_7.grib
processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/2m_dewpoint_temperature/2m_dewpoint_temperature_2019_7.grib
Adiabatic Sharpener will be applied


/home/potzschf/repos/helperToolz/guzinski.py:361: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(block, axis = 2)


processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/2m_temperature/2m_temperature_2019_7.grib
Adiabatic Sharpener will be applied


/home/potzschf/repos/helperToolz/guzinski.py:361: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(block, axis = 2)


processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/surface_pressure/surface_pressure_2019_7.grib
Barometric Sharpener will be applied


/home/potzschf/repos/helperToolz/guzinski.py:636: RuntimeWarning: divide by zero encountered in divide
  ratio = (era5_corrected_Temp - STANDARD_ADIABAT * delta_z) / era5_corrected_Temp
/home/potzschf/repos/helperToolz/guzinski.py:357: RuntimeWarning: invalid value encountered in subtract
  vals = before - (before - after) * (np.array(arr_min, dtype=np.float16) / 60)
/home/potzschf/repos/helperToolz/guzinski.py:361: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(block, axis = 2)


processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/surface_solar_radiation_downward_clear_sky/surface_solar_radiation_downward_clear_sky_2019_7.grib


/home/potzschf/repos/helperToolz/guzinski.py:523: RuntimeWarning: All-NaN slice encountered
  ssrd = np.nanmax(block, axis = 2)


In [7]:
# bring all variables down to 20m
wind100_u_20 = warp_np_to_reference(wind100_u, LST_acq_spatial_sub, LST_file)
wind100_v_20 = warp_np_to_reference(wind100_v, LST_acq_spatial_sub, LST_file)
ssrd_20 = warp_np_to_reference(ssrd, LST_acq_spatial_sub, LST_file)
air_temp_20 = warp_np_to_reference(air_temp, LST_acq_spatial_sub, LST_file)
dew_temp_20 = warp_np_to_reference(dew_temp, LST_acq_spatial_sub, LST_file)
sp_20 = warp_np_to_reference(sp, LST_acq_spatial_sub, LST_file)
szenith_20 = warp_np_to_reference(szenith, LST_acq_spatial_sub, LST_file)
sazimuth_20 = warp_np_to_reference(sazimuth, LST_acq_spatial_sub, LST_file)

# calculate windspeed
wind_speed_20 = calc_wind_speed(wind100_u_20, wind100_v_20)

# load vza
vza_ds = warp_raster_to_reference(VZA_at_acq_file_sub, LST_file, output_path='MEM', resampling='bilinear')
vza_20 = vza_ds.GetRasterBand(1).ReadAsArray()
# load sharpened LST
lst_ds = gdal.Open(LST_file)
lst_20 =lst_ds.GetRasterBand(1).ReadAsArray()

del wind100_u, wind100_v, ssrd, air_temp, dew_temp, sp, wind100_u_20, wind100_v_20, szenith, sazimuth

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
# power mask 
powerMask = np.logical_and.reduce(arr > 0 for arr in [ssrd_20, air_temp_20, dew_temp_20, sp_20, szenith_20, sazimuth_20, wind_speed_20, lst_20, vza_20])
ssrd_20 = np.ma.masked_array(ssrd_20, mask=powerMask)
air_temp_20 = np.ma.masked_array(air_temp_20, mask=powerMask)
dew_temp_20 = np.ma.masked_array(dew_temp_20, mask=powerMask)
sp_20 = np.ma.masked_array(sp_20, mask=powerMask)
szenith_20 = np.ma.masked_array(szenith_20, mask=powerMask)
sazimuth_20 = np.ma.masked_array(sazimuth_20, mask=powerMask)
wind_speed_20 = np.ma.masked_array(wind_speed_20, mask=powerMask)
lst_20 = np.ma.masked_array(lst_20, mask=powerMask)
vza_20 = np.ma.masked_array(vza_20, mask=powerMask)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: create 10 arrays with random values
arrays = [ssrd_20, air_temp_20, dew_temp_20, szenith_20, sazimuth_20, wind_speed_20, lst_20, vza_20]# , sp_20
names = ['SSRD', 'TEMP', 'DEW', 'ZENITH', 'AZIMUTH', 'WINDSPEED', 'LST', 'VZA']#'SP',

flattened = []
valid_names = []

for name, arr in zip(names, arrays):
    vals = np.ma.compressed(arr) if np.ma.isMaskedArray(arr) else arr.ravel()
    vals = vals[~np.isnan(vals)]  # remove NaNs
    if vals.size > 0:
        flattened.append(vals)
        valid_names.append(name)
    else:
        print(f"⚠️ Skipping {name} — all values masked or NaN")

# Histograms
plt.figure(figsize=(10, 6))
for i, data in enumerate(flattened):
    plt.hist(data, bins=50, alpha=0.4, label=valid_names[i])
plt.legend()
plt.title("Value Distributions - Histograms")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

# Boxplot
plt.figure(figsize=(10, 6))
plt.boxplot(flattened, labels=valid_names, vert=True)
plt.xticks(rotation=45)
plt.title("Value Distributions - Boxplot")
plt.ylabel("Value")
plt.show()

# Violin plot
plt.figure(figsize=(10, 6))
plt.violinplot(flattened, showmeans=True)
plt.xticks(np.arange(1, len(valid_names) + 1), valid_names, rotation=45)
plt.title("Value Distributions - Violin Plot")
plt.ylabel("Value")
plt.show()

In [ ]:
np.max(ssrd_20.filled)

In [ ]:
# calculate the NDVI from the S2 composite (following formula from force --> bandnames: (NIR - RED) / (NIR + RED))
S2_ds = gdal.Open(S2_file)
for idx, bname in enumerate(getBandNames(S2_file)):
    if bname == 'RED':
        red = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    elif bname == 'NIR':
        nir = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    else:
        continue
ndvi_20 = (nir - red) / (nir + red)
ndvi_20_ma = np.ma.masked_invalid(ndvi_20)
ndvi_20_ma = np.ma.masked_where(ndvi_20_ma < 0, ndvi_20_ma)
LAI_np = 0.57*np.exp(2.33*ndvi_20)
LAI_pos = np.ma.masked_where(LAI_np < 0, LAI_np)

# estimate canopy height from estimated LAI
hc = hc_from_lai(LAI_pos, hc_max = 1.2, lai_max = np.nanmax(LAI_np), hc_min=0)

# estimate long wave irradiance
ea = meteo_utils.calc_vapor_pressure(T_K=dew_temp_20)
L_dn = calc_longwave_irradiance(ea = ea, t_a_k = air_temp_20, p = sp_20, z_T = 100, h_C = hc) # ## does that make sense with the 100m!!!!!!!!!!!!!!!!!!!
d_0_0 = resistances.calc_d_0(h_C=hc)
z_0 = resistances.calc_z_0M(h_C=hc)

In [ ]:
# calculate shortwave radiation of soil and canopy
difvis, difnir, fvis, fnir = net_radiation.calc_difuse_ratio(S_dn = ssrd_20, sza = np.mean(szenith_20))

skyl = difvis * fvis + difnir * fnir
S_dn_dir = ssrd_20 * (1.0 - skyl)
S_dn_dif = ssrd_20 * skyl

# Leaf spectral properties:{rho_vis_C: visible reflectance, tau_vis_C: visible transmittance, rho_nir_C: NIR reflectance, tau_nir_C: NIR transmittance}
rho_vis_C=np.full(LAI_pos.shape, 0.05, np.float32)
tau_vis_C=np.full(LAI_pos.shape, 0.08, np.float32)
rho_nir_C=np.full(LAI_pos.shape, 0.32, np.float32)
tau_nir_C=np.full(LAI_pos.shape, 0.33, np.float32) 

# Soil spectral properties:{rho_vis_S: visible reflectance, rho_nir_S: NIR reflectance}
rho_vis_S=np.full(LAI_pos.shape, 0.07, np.float32)
rho_nir_S=np.full(LAI_pos.shape, 0.25, np.float32)

# F = local LAI
F = LAI_pos / 1
# calculate clumping index
Omega0 = clumping_index.calc_omega0_Kustas(LAI = LAI_np, f_C = 1, x_LAD=1)
Omega = clumping_index.calc_omega_Kustas(Omega0, np.mean(szenith_20))
LAI_eff = F * Omega

Sn_C, Sn_S = net_radiation.calc_Sn_Campbell(lai = LAI_pos, sza = np.mean(szenith_20), S_dn_dir = S_dn_dir, S_dn_dif = S_dn_dif, fvis = fvis,
                                    fnir = fnir, rho_leaf_vis = rho_vis_C, tau_leaf_vis = tau_vis_C, rho_leaf_nir = rho_nir_C, 
                                    tau_leaf_nir = tau_nir_C, rsoilv = rho_vis_S, rsoiln = rho_nir_S, x_LAD=1, LAI_eff=LAI_eff)

# calculate other roughness stuff
z_0M, d = resistances.calc_roughness(LAI=np.nanmean(LAI_pos), h_C=hc, w_C=1, landcover=11, f_c=None)
z_0M_array = np.full(LAI_pos.shape, z_0M)
d_array = np.full(LAI_pos.shape, d)
fg = calc_fg_gutman(ndvi = ndvi_20_ma, ndvi_min = np.nanmin(ndvi_20), ndvi_max = np.nanmax(ndvi_20))

In [ ]:
emis_C = 0.98
emis_S = 0.95
h_C = hc 
z_u = 100
z_T = 100

In [ ]:
output = TSEB.TSEB_PT(lst_20, vza_20, air_temp_20, wind_speed_20, ea, sp_20, Sn_C, Sn_S, L_dn, LAI_pos, h_C, emis_C, emis_S, 
                      z_0M, z_0M_array, z_u, z_T, resistance_form=None, calcG_params=None, const_L=None, 
                      kB=0.0, massman_profile=None, verbose=True)

len(output)
ld = output[6]/ssrd_20
heat_latent_scaled = ssrd_20 * ld
et_daily_p = TSEB.met.flux_2_evaporation(heat_latent_scaled, t_k=air_temp_20, time_domain=24)

In [ ]:
output_file = f'{tempOut}test1.tif'
# arid_aoi: 
# Get the metadata from the NDVI raster
lst_ras = rasterio.open(LST_file)
LST_metadata = lst_20.meta

# et_daily_p = np.ma.masked_where(et_daily < 0, et_daily)
# et_daily_p = et_daily_p.filled(np.nan)  # Fill masked values with np.nan

# Create the destination metadata
dst_metadata = {
    'driver': 'GTiff',
    'dtype': et_daily_p.dtype,  # Use the dtype of the masked array
    'nodata': np.nan,  # Set nodata to np.nan
    'width': et_daily_p.shape[1],
    'height': et_daily_p.shape[0],
    'count': 1,  # Assuming a single-band GeoTIFF
    'crs': LST_metadata['crs'],
    'transform': LST_metadata['transform'],
    'compress': 'lzw'
}
        
# Save the array as a GeoTIFF
with rasterio.open(output_file, 'w', **dst_metadata) as dst:
    dst.write(et_daily_p, 1) 